Cargo los datos sobre temperaturas durante el año 2020 de Barcelona desde la estación de Drassanes que tiene un criterio de temperatura a nivel ciudad.

In [2]:
import pandas as pd

# Cargo los archivos CSV
df1 = pd.read_csv('temp_012020_062020.csv')
df2 = pd.read_csv('temp_072020_122020.csv')

# Concateno los DataFrames por filas
df_temp_2020 = pd.concat([df1, df2], axis=0, ignore_index=True)

df_temp_2020.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
0,2020-01-01,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"8,9","0,0","5,0",07:20,"12,8",13:00,17.0,"1,7","4,4",13:20,79,89,Varias,67,13:00
1,2020-01-02,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"9,2","0,0","4,4",06:00,"13,9",14:50,28.0,"3,1","7,5",12:40,81,92,Varias,66,Varias
2,2020-01-03,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"10,5","0,0","7,1",07:20,"13,9",13:30,34.0,"1,9","6,7",08:30,76,89,04:50,59,14:30
3,2020-01-04,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"10,2","0,0","4,7",06:20,"15,8",13:20,30.0,"2,8","7,8",16:20,62,82,06:20,49,15:30
4,2020-01-05,0201D,"BARCELONA, PORT OL?MPIC",BARCELONA,26,"11,0","0,0","7,0",07:30,"15,1",13:20,9.0,"1,4","7,8",15:00,67,89,21:30,47,11:40


El dataframe está compuesto por 365 filas y 20 columnas

In [3]:
df_temp_2020.shape 

(366, 20)

Hay columnas que no contemplo utilizar por lo que decido eliminarlas para aligerar el dataframe.

In [4]:
df_temp_2020= df_temp_2020.drop(columns=['indicativo', 'nombre', 'hrMax', 'horaHrMax', 'hrMin', 'horaHrMin', 'altitud', 'dir', 'racha', 'horaracha', 'horatmin', 'horatmax'])

df_temp_2020.head()

,fecha,provincia,tmed,prec,tmin,tmax,velmedia,hrMedia
0,2020-01-01,BARCELONA,"8,9","0,0","5,0","12,8","1,7",79
1,2020-01-02,BARCELONA,"9,2","0,0","4,4","13,9","3,1",81
2,2020-01-03,BARCELONA,"10,5","0,0","7,1","13,9","1,9",76
3,2020-01-04,BARCELONA,"10,2","0,0","4,7","15,8","2,8",62
4,2020-01-05,BARCELONA,"11,0","0,0","7,0","15,1","1,4",67


Reviso las características de cada columna, observo varios detalles sobre ellas que considero tratar:

* `fecha` es del tipo `object` y el formato debería ser `datetime`
* Hay varias columas que aparecen como tipo `object` al utiizar una coma como decimal y las transformaré como float con punto.


In [5]:
df_temp_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fecha      366 non-null    object
 1   provincia  366 non-null    object
 2   tmed       366 non-null    object
 3   prec       366 non-null    object
 4   tmin       366 non-null    object
 5   tmax       366 non-null    object
 6   velmedia   364 non-null    object
 7   hrMedia    366 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 23.0+ KB


Realizo la transformación de la columna `fecha` al tipo `datetime`

In [6]:
df_temp_2020['fecha']= pd.to_datetime(df_temp_2020['fecha'], format='%Y-%m-%d')
print(df_temp_2020.dtypes)

fecha        datetime64[ns]
provincia            object
tmed                 object
prec                 object
tmin                 object
tmax                 object
velmedia             object
hrMedia               int64
dtype: object


Transformo las columnas 'tmed', 'prec', 'tmin', 'tmax' al tipo float, modificando el símbolo de coma por punto.

In [7]:
columnas_a_float = ['tmed', 'prec', 'tmin', 'tmax', 'velmedia']

# Reemplazo las comas por puntos para que pueda realizarse el cambio al tipo float
for col in columnas_a_float:
    df_temp_2020[col] = df_temp_2020[col].str.replace(',', '.')  

df_temp_2020[columnas_a_float] = df_temp_2020[columnas_a_float].astype(float)

print(df_temp_2020.dtypes)

fecha        datetime64[ns]
provincia            object
tmed                float64
prec                float64
tmin                float64
tmax                float64
velmedia            float64
hrMedia               int64
dtype: object


In [8]:
df_temp_2020.isnull().sum()

fecha        0
provincia    0
tmed         0
prec         0
tmin         0
tmax         0
velmedia     2
hrMedia      0
dtype: int64

Analizo los valores nulos de la variable `velmedia`; observo que aparecen varias columnas en ese registros con valores faltantes.

In [9]:
nulos_tmed = df_temp_2020['velmedia'].isnull()
df_temp_2020[nulos_tmed]

,fecha,provincia,tmed,prec,tmin,tmax,velmedia,hrMedia
282,2020-10-09,BARCELONA,20.7,0.6,17.9,23.5,NaN,91
283,2020-10-10,BARCELONA,17.0,41.0,13.4,20.6,NaN,92


La idea de la imputación en este caso es mantener la continuidad de la serie. Imputo los valores faltantes utilizando una función, que en caso de que encuentre un valor nulo en una columna, buscará el valor correspondiente en la fila anterior (día anterior). Si el valor anterior existe, se utiliza para reemplazar el valor nulo. 

In [10]:
# Ordeno el DataFrame por fecha para asegurar la correcta imputación
df_temp_2020 = df_temp_2020.sort_values('fecha')

# Selecciono las columnas a imputar 
columnas_a_imputar = ['velmedia']

# Se itera sobre las filas y las columnas a imputar
for index, row in df_temp_2020.iterrows():
    for columna in columnas_a_imputar:
        if pd.isna(row[columna]):
            # Buscar el valor del día anterior en la misma columna
            valor_anterior = df_temp_2020.loc[index - 1, columna]
            # Imputar si el valor anterior existe
            if not pd.isna(valor_anterior):
                df_temp_2020.loc[index, columna] = valor_anterior

Compruebo que los valores ha sido imputados totalmente.

In [11]:
df_temp_2020.isnull().sum()

fecha        0
provincia    0
tmed         0
prec         0
tmin         0
tmax         0
velmedia     0
hrMedia      0
dtype: int64

- Renombro las columna `provincia` por `ciudad`.

In [12]:
df_temp_2020= df_temp_2020.rename(columns={'provincia': 'ciudad', 'hrMedia': 'hrmedia'})
df_temp_2020= df_temp_2020[['fecha', 'ciudad', 'tmin', 'tmax', 'tmed', 'prec','hrmedia', 'velmedia']]
df_temp_2020.head()


,fecha,ciudad,tmin,tmax,tmed,prec,hrmedia,velmedia
0,2020-01-01,BARCELONA,5.0,12.8,8.9,0.0,79,1.7
1,2020-01-02,BARCELONA,4.4,13.9,9.2,0.0,81,3.1
2,2020-01-03,BARCELONA,7.1,13.9,10.5,0.0,76,1.9
3,2020-01-04,BARCELONA,4.7,15.8,10.2,0.0,62,2.8
4,2020-01-05,BARCELONA,7.0,15.1,11.0,0.0,67,1.4


Codifico la columna `ciudad` utilizando el valor `1` para poder relacionarlo en la base de datos con la tabla que corresponda a las ciudades

In [13]:
mapeo_ciudad= {"BARCELONA": "1"}

df_temp_2020['ciudad']= df_temp_2020['ciudad'].map(mapeo_ciudad)
df_temp_2020.head()

,fecha,ciudad,tmin,tmax,tmed,prec,hrmedia,velmedia
0,2020-01-01,1,5.0,12.8,8.9,0.0,79,1.7
1,2020-01-02,1,4.4,13.9,9.2,0.0,81,3.1
2,2020-01-03,1,7.1,13.9,10.5,0.0,76,1.9
3,2020-01-04,1,4.7,15.8,10.2,0.0,62,2.8
4,2020-01-05,1,7.0,15.1,11.0,0.0,67,1.4


Exporto el dataframe a un archivo en formato `csv`

In [14]:
df_temp_2020.to_csv('temp_bcn_2020.csv', index= False, encoding='utf-8')

Resumen del dataframe creado:

- Se eliminaron las columnas 'indicativo', 'nombre', 'hrMax', 'horaHrMax', 'hrMin', 'horaHrMin', 'altitud', 'dir', 'racha', 'horaracha', 'horatmin', 'horatmax'.
- Se realizaron transformaciones de tipo en las columnas 'tmed', 'prec', 'tmin', 'tmax', 'velmedia'.
- Se transformó la columna 'fecha' a tipo datetime
- Se trataron los valores nulos mediante la imputación del registro del día anterior al faltante, buscando mantener la continuidad de la serie. No hay valores faltantes.
- Se renombró la columna 'provincia' por 'ciudad' y se asignó el valor 1 para que sea utilizado en un modelo de tablas relaciones como identificador(id).
- Se exporto el dataframe a un archivo en formato csv.